In [1]:
# import the libraries
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
# Re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'C:/Users/Madhur/Desktop/Skin burn analysis (Hackathon)/train'
valid_path = 'C:/Users/Madhur/Desktop/Skin burn analysis (Hackathon)/test'

In [3]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 7s 0us/step


In [4]:
# don't train existing weights
for layer in resnet.layers:
    layer.trainable = False

In [5]:
  # useful for getting number of output classes
folders = glob('train')

In [6]:
folders

['train']

In [7]:
# our layers - you can add more if you want
x = Flatten()(resnet.output)

In [8]:
prediction = Dense(3, activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [9]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [10]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('C:/Users/Madhur/Desktop/Skin burn analysis (Hackathon)/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 920 images belonging to 3 classes.


In [13]:
test_set = test_datagen.flow_from_directory('C:/Users/Madhur/Desktop/Skin burn analysis (Hackathon)/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 540 images belonging to 3 classes.


In [14]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=100,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/100
29/29 [==============================] - 67s 2s/step - loss: 3.2180 - accuracy: 0.4402 - val_loss: 0.9996 - val_accuracy: 0.5926
Epoch 2/100
29/29 [==============================] - 64s 2s/step - loss: 1.1094 - accuracy: 0.5033 - val_loss: 0.8463 - val_accuracy: 0.6407
Epoch 3/100
29/29 [==============================] - 66s 2s/step - loss: 1.0529 - accuracy: 0.5533 - val_loss: 0.9457 - val_accuracy: 0.5981
Epoch 4/100
29/29 [==============================] - 66s 2s/step - loss: 1.0327 - accuracy: 0.5337 - val_loss: 1.0072 - val_accuracy: 0.3833
Epoch 5/100
29/29 [==============================] - 65s 2s/step - loss: 0.9549 - accuracy: 0.5641 - val_loss: 0.8009 - val_accuracy: 0.6500
Epoch 6/100
29/29 [==============================] - 63s 2s/step - loss: 0.9880 - accuracy: 0.5652 - val_loss: 1.1286 - val_accuracy: 0.3185
Epoch 7/100
29/29 [==============================] - 63s 2s/step - loss: 1.2132 - accuracy: 0.5087 - val_loss: 0.9506 - val_accuracy: 0.6241
Epoch 8/100
2

In [ ]:
#Accuracy of model: 71.96%